# Async (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

Installing Packages Expecto

Installing Packages Expecto.

Installing Packages Expecto..

Installing Packages Expecto...

Installing Packages Expecto....

Installing Packages Expecto.....

Installing Packages Expecto......

Installed Packages Expecto, 10.1.0

In [ ]:
#!import ../nbs/Common.fs

In [ ]:
open Common

## runWithTimeout

In [ ]:
let runWithTimeout timeout fn =
    try
        Async.RunSynchronously (fn, timeout) |> Some
    with
    | :? System.TimeoutException as ex ->
        let info = ex

        let getLocals () = $"timeout: {timeout} / exception: {ex.Message} / {getLocals ()}"
        trace Debug (fun () -> "runWithTimeout") getLocals
        None
    | e -> raise e

In [ ]:
//// test

Async.Sleep 50
|> runWithTimeout 10
|> _equal None

19:44:04 #1 [Debug] runWithTimeout / timeout: 10 / exception: The operation has timed out.
<null>


In [ ]:
//// test

Async.Sleep 10
|> runWithTimeout 50
|> _equal (Some ())

FSharpOption<Unit>
      Value: <null>
